In [1]:
import sys,os
#import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import torch.utils
#import cv2

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from random import randint

In [4]:
device = torch.device("cuda")
#device = torch.device("cpu")
print(device)

cuda


In [5]:
path = 'C:/Users/vigne/Desktop/age_gender_folder'


In [6]:
x_train = np.load(path+'/x_train.npy')
y_train = np.load(path+'/y_train.npy')
x_val = np.load(path+'/x_val.npy')
y_val = np.load(path+'/y_val.npy')

In [7]:
x_train.shape

(14952, 1, 48, 48)

In [8]:
x_val.shape

(1662, 1, 48, 48)

In [9]:
#size = 224
transform = transforms.Compose(
    [transforms.ToPILImage(),
     #transforms.Resize((size,size)),
     transforms.RandomRotation(degrees = 20),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5),(0.5))
])

class MyDataset(Dataset):
    def __init__(self, data, target, transform=transform):
        self.data = torch.from_numpy(data).float()
        self.target = torch.from_numpy(target).long()
        self.transform = transform
     
    def __len__(self):
        return len(self.target)
    
    def __getitem__(self, index):
        
        return self.data[index], self.target[index]
        

In [10]:
class myModel(nn.Module):
    def __init__(self):
        super(myModel , self).__init__()
     
        self.conv1 = nn.Conv2d(1,  16,  kernel_size=3 )
        self.pool1  = nn.MaxPool2d(2,2)
        self.batch1 = nn.BatchNorm2d(16)

        self.conv2 = nn.Conv2d(16,  32, kernel_size=3 )
        self.pool2  = nn.MaxPool2d(2,2)
        self.batch2 = nn.BatchNorm2d(32)
      
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3 )
        self.pool3  = nn.MaxPool2d(2,2)
        self.batch3 = nn.BatchNorm2d(64)
    
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3 )
        self.pool4  = nn.MaxPool2d(2,2)
        self.batch4 = nn.BatchNorm2d(128)

        self.linear1 = nn.Linear(128*1*1, 64)
        self.linear2 = nn.Linear(64,32)
        self.linear3 = nn.Linear(32,16)
        self.dropout1 = nn.Dropout(p=0.2)
        self.dropout2 = nn.Dropout(p=0.4)
        
    
    def forward(self, x):

        x = self.conv1(x)
        x = F.relu(x)
        x = self.batch1(x)
        x = self.pool1(x)
    
        x = self.conv2(x)
        x = F.relu(x)
        x = self.batch2(x)
        x = self.pool2(x)
    
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool3(x)
    
        x = self.conv4(x)
        x = F.relu(x)
        x = self.pool4(x)
        x = self.batch4(x)

        x = x.view(-1, 128)

        x = self.linear1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.linear2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.linear3(x)

        
        return x

In [34]:
batchsize = 100
in_channel = 1
num_classes = 16
learning_rate = 0.08
num_epochs = 30

In [24]:
train_dataset = MyDataset(x_train, y_train)
train_loader = DataLoader(dataset=train_dataset, batch_size=batchsize, shuffle=True)

val_dataset = MyDataset(x_val, y_val)
val_loader = DataLoader(dataset=val_dataset, batch_size=batchsize, shuffle=True)

In [13]:
len(train_loader)

234

In [39]:
def eval_on_val_set(val_acc,val_loss,val_loader,val_dataset):

    running_loss=0
    running_acc=0
    num_batches=0
    model.eval()
    for (data, targets) in val_loader:

        data=data.to(device)
        targets=targets.to(device)

        scores = model(data) 

        loss = criterion(scores, targets)

        running_loss += loss.detach().item()*data.size(0)
        _, predictions = torch.max(scores.data, 1)
        running_acc += (predictions == targets).sum().item()

        num_batches += 1

    lo =  running_loss/len(val_dataset)
    acc = 100*running_acc/len(val_dataset)
    
    val_acc.append(acc)
    val_loss.append(loss)
    
    print('val loss = {}  val accuracy = {}'.format(lo,acc))
    
    return val_acc,val_loss

In [41]:
model = myModel()

In [42]:
model = model.to(device)

In [43]:
criterion = nn.CrossEntropyLoss()


In [44]:
losses=[]
accuracy=[]

val_loss = []
val_acc = []

for epoch in range(25):
    model.train()
    running_loss=0
    running_acc=0
    num_batches=0
    
    # divide the learning rate by 2 at epoch 10, 14 and 18
    if epoch%4==0 and epoch!=0:
        learning_rate = learning_rate / 2

   # create a new optimizer at the beginning of each epoch: give the current learning rate.   
    optimizer=torch.optim.SGD( model.parameters() , lr=learning_rate )
    
    for count, (data, targets) in enumerate(train_loader):
        num_batches += 1
        optimizer.zero_grad()
        data.requires_grad_()
        
        data = data.to(device)
        targets = targets.to(device)
        
        scores = model(data)
        
        loss = criterion(scores, targets)
        running_loss += loss.detach().item()*data.size(0)
        
        loss.backward()
        optimizer.step()
        
        _, predictions = scores.max(1)
        running_acc += (predictions == targets).float().sum()
    
    lo = running_loss/len(train_dataset)
    acc = 100*running_acc/len(train_dataset)
    losses.append(lo)
    accuracy.append(acc)
    print('Epoch {} : loss = {}  accuracy = {}'.format(epoch,lo,acc))
    
    if epoch%5==0:
        val_acc,val_loss = eval_on_val_set(val_acc,val_loss,val_loader,val_dataset)

Epoch 0 : loss = 2.657688318885795  accuracy = 17.803638458251953
val loss = 2.4957652384623725  val accuracy = 27.79783393501805
Epoch 1 : loss = 2.4270234356084694  accuracy = 26.685392379760742
Epoch 2 : loss = 2.2887495458923346  accuracy = 30.557783126831055
Epoch 3 : loss = 2.148403067267343  accuracy = 35.55377197265625
Epoch 4 : loss = 2.0244913206437234  accuracy = 39.80069351196289
Epoch 5 : loss = 1.9440834954054105  accuracy = 41.90743637084961
val loss = 1.8821188948788512  val accuracy = 42.29843561973526
Epoch 6 : loss = 1.8787572246429562  accuracy = 43.82691192626953
Epoch 7 : loss = 1.813048628895853  accuracy = 45.05751419067383
Epoch 8 : loss = 1.7726387141731859  accuracy = 45.99384689331055
Epoch 9 : loss = 1.7454328442908278  accuracy = 46.63589859008789
Epoch 10 : loss = 1.7152223632959456  accuracy = 47.30470657348633
val loss = 1.6970957925029897  val accuracy = 47.89410348977136
Epoch 11 : loss = 1.6929082778636755  accuracy = 48.13402557373047
Epoch 12 : los

In [ ]:
plt.plot(losses,'-o')
plt.title('Loss per epoch for 30 epochs')
plt.ylabel('Loss')
plt.xlabel('No. of epochs')

In [ ]:
plt.plot(accuracy,'-o')
plt.title('Accuracy per epoch for 30 epochs')
plt.ylabel('Accuracy')
plt.xlabel('No. of epochs')

In [ ]:
#eval_on_val_set()

In [ ]:
import pickle as pk
dest_path = "/content/drive/My Drive/cv_project/model.pkl"
#model = myModel()
pk.dump(model.to('cpu'),open(dest_path,'wb'))